<a href="https://colab.research.google.com/github/mmcncc/ipynb/blob/gg/Z_Image_Turbo_4bit_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 单元格 1: 安装依赖
!pip install -U git+https://github.com/huggingface/diffusers git+https://github.com/Disty0/sdnq
!pip install flask pyngrok

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-dxy5i5x5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-dxy5i5x5
  Resolved https://github.com/huggingface/diffusers to commit be3c2a0667493022f17d756ca3dba631d28dfb40
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/Disty0/sdnq to /tmp/pip-req-build-ufs48otp
  Running command git clone --filter=blob:none --quiet https://github.com/Disty0/sdnq /tmp/pip-req-build-ufs48otp
  Resolved https://github.com/Disty0/sdnq to commit 6a5be4441c3a25138214eca58270da61c88b65d4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-py3-none-any.whl size=4636030 sha256=6618d8bb6bddc94453abce62f71bd9463776a67b003

In [2]:
# 单元格 2: 加载模型（需要几分钟）
import torch
import diffusers
from sdnq import SDNQConfig
from sdnq.loader import apply_sdnq_options_to_model

pipe = diffusers.ZImagePipeline.from_pretrained(
    "Disty0/Z-Image-Turbo-SDNQ-uint4-svd-r32",
    torch_dtype=torch.float32,
    device_map="cuda"
)
pipe.transformer = apply_sdnq_options_to_model(pipe.transformer, use_quantized_matmul=True)
pipe.text_encoder = apply_sdnq_options_to_model(pipe.text_encoder, use_quantized_matmul=True)

print("模型加载完成！")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

scheduler_config.json:   0%|          | 0.00/487 [00:00<?, ?B/s]

...

transformer/diffusion_pytorch_model.safe(…):   0%|          | 0.00/3.48G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


模型加载完成！


In [ ]:
# 单元格 3: 启动 API 服务
# 运行后会显示一个 Gradio 地址，复制到本地脚本中使用
import io
import base64
import random
import gradio as gr
import torch
import PIL.Image

# 图像生成函数，适配 Gradio 接口
def sdnq_image_generator(prompt: str, width: int, height: int, seed: int) -> PIL.Image.Image:
    print(f"收到请求: {prompt[:50]}...")

    # 生成图片
    image = pipe(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=9,
        guidance_scale=0.0,
        generator=torch.manual_seed(seed),
    ).images[0]

    print("图片生成成功！")
    return image

# 创建 Gradio 接口
iface = gr.Interface(
    fn=sdnq_image_generator,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="A vibrant, fantastical landscape..."),
        gr.Slider(minimum=256, maximum=1024, step=64, value=1024, label="Width"),
        gr.Slider(minimum=256, maximum=1024, step=64, value=1024, label="Height"),
        gr.Number(label="Seed", value=random.randint(0, 999999), precision=0)
    ],
    outputs=gr.Image(label="Generated Image", type="pil"),
    title="Z-Image-Turbo-SDNQ-uint4-svd-r32 Image Generator",
    description="Generate high-quality images with Z-Image-Turbo-SDNQ-uint4-svd-r32 model."
)

# 启动 Gradio 接口，并使用 share=True 创建公共链接
# Gradio 会自动处理外部访问，无需 ngrok
print("正在启动 Gradio 界面...")
iface.launch(share=True, debug=True)
print("Gradio 界面已启动！请查看上面的公共链接。")

正在启动 Gradio 界面...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://347dfe0ab25ab530bf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


收到请求: 一张电影感十足的8k摄影杰作，画面充满治愈感和青春活力。构图采用黄金分割，一个穿着白色碎花裙的青春少...


/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
W1210 10:41:08.644000 799 torch/_inductor/utils.py:1558] [2/0] Not enough SMs to use max_autotune_gemm mode


  0%|          | 0/9 [00:00<?, ?it/s]

图片生成成功！
收到请求: 一张电影感十足的8k摄影杰作，画面充满治愈感和青春活力。构图采用黄金分割，一个穿着白色碎花裙的青春少...


...

图片生成成功！
收到请求: 一张电影感十足的8k摄影杰作，画面充满治愈感和青春活力。构图采用黄金分割，一个穿着白色碎花裙的青春少...


  0%|          | 0/9 [00:00<?, ?it/s]

图片生成成功！
收到请求: 一张电影感十足的8k摄影杰作，画面充满治愈感和青春活力。构图采用黄金分割，一个穿着白色碎花裙的青春少...


  0%|          | 0/9 [00:00<?, ?it/s]

图片生成成功！
